# Notebook to select host galaxies and submit candidates to TNS

In [1]:
import sys, os
import pandas as pd
import numpy as np
import ipyaladin as ipyal
from ipywidgets import Layout, Box, widgets

In [2]:
from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, Column

In [3]:
sys.path.append("./lib")
from alerce_tns import *

In [4]:
# continuously load modules (for easier debugging)
#%load_ext autoreload
#%autoreload 2

# Start API

In [5]:
alerce = alerce_tns()

# Load recent candidates

# CHANGE DATE

In [6]:
refstring = "20200130"
candidates_dir = "candidates"
candidates = open("%s/%s.csv" % (candidates_dir, refstring)).read().splitlines()
print(candidates, len(candidates))

['ZTF20aakoeur', 'ZTF20aakodiq', 'ZTF20aaknzsj', 'ZTF20aakbtyz', 'ZTF20aakmbql', 'ZTF20aakocfl', 'ZTF20aakoenh', 'ZTF20aaknyfo'] 8


# Select host galaxies using aladin widget

In [7]:
alerce.select_hosts(candidates, catsHTM=False, vizier=False)

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Cannot connect with Simbad...


...

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

/home/fforster/anaconda3/lib/python3.6/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '15:24:19.1979 +61:54:30.5928': No astronomical object found :
  (error.line, error.msg))


Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

/home/fforster/anaconda3/lib/python3.6/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '12:40:44.5753 +60:20:24.9724': No astronomical object found :
  (error.line, error.msg))


In [8]:
alerce.candidate_host_names

{'ZTF20aakoeur': 'WISEA J160553.79+382502.3',
 'ZTF20aakodiq': 'WISEA J150439.71+355654.5',
 'ZTF20aaknzsj': 'NGC 6091',
 'ZTF20aakbtyz': 'SDSS J114219.14+445745.5',
 'ZTF20aakmbql': 'SDSS J092332.59+225622.0',
 'ZTF20aakocfl': 'WISEA J152419.36+615427.4',
 'ZTF20aakoenh': 'WISEA J160709.26+693215.8',
 'ZTF20aaknyfo': 'SDSS J124044.98+602032.3'}

In [9]:
alerce.candidate_host_redshifts

{'ZTF20aakoeur': 0.286071,
 'ZTF20aakodiq': 0.045943,
 'ZTF20aaknzsj': 0.022979,
 'ZTF20aakbtyz': '0.090',
 'ZTF20aakmbql': '0.213',
 'ZTF20aakocfl': 'nan',
 'ZTF20aakoenh': 'nan',
 'ZTF20aaknyfo': '0.120'}

# Manual modifications

In [10]:
#oid = "ZTF20aakdppm" #ZTF19acxrihd"
#alerce.candidate_host_names[oid] = "M31"
#alerce.candidate_host_redshifts[oid] = "nan"

# Clean nans

In [11]:
for k in list(alerce.candidate_host_redshifts.keys()):
    if alerce.candidate_host_redshifts[k] == "nan":
        alerce.candidate_host_redshifts.pop(k)

# Save hosts

In [12]:
df_host_names = pd.DataFrame.from_dict(alerce.candidate_host_names, orient='index', columns=["host_id"])
df_host_names.index.name='oid'
df_host_redshifts = pd.DataFrame.from_dict(alerce.candidate_host_redshifts, orient='index', columns=["z"])
df_host_redshifts.index.name='oid'
df_hosts = df_host_names.join(df_host_redshifts)
df_hosts.to_csv("candidates/%s_hosts.csv" % refstring)
df_hosts.head(100)

,host_id,z
oid,,
ZTF20aakoeur,WISEA J160553.79+382502.3,0.286071
ZTF20aakodiq,WISEA J150439.71+355654.5,0.045943
ZTF20aaknzsj,NGC 6091,0.022979
ZTF20aakbtyz,SDSS J114219.14+445745.5,0.090
ZTF20aakmbql,SDSS J092332.59+225622.0,0.213
ZTF20aakocfl,WISEA J152419.36+615427.4,NaN
ZTF20aakoenh,WISEA J160709.26+693215.8,NaN
ZTF20aaknyfo,SDSS J124044.98+602032.3,0.120
ZTF20aakdppm,M31,NaN


# Display host galaxy properties

In [13]:
for oid in candidates:
    if oid in alerce.candidate_host_redshifts.keys():
        print(oid, alerce.candidate_host_names[oid], alerce.candidate_host_redshifts[oid])
    elif oid in alerce.candidate_host_names.keys():
        print(oid, alerce.candidate_host_names[oid])
    else:
        alerce.candidate_host_names[oid] = ""
        alerce.candidate_host_redshifts[oid] = ""
        print(oid)

ZTF20aakoeur WISEA J160553.79+382502.3 0.286071
ZTF20aakodiq WISEA J150439.71+355654.5 0.045943
ZTF20aaknzsj NGC 6091 0.022979
ZTF20aakbtyz SDSS J114219.14+445745.5 0.090
ZTF20aakmbql SDSS J092332.59+225622.0 0.213
ZTF20aakocfl WISEA J152419.36+615427.4
ZTF20aakoenh WISEA J160709.26+693215.8
ZTF20aaknyfo SDSS J124044.98+602032.3 0.120


# Sandbox or real

In [14]:
sandbox = False # True: test in sandbox, False: send to real website

# Get API key and url

In [15]:
if sandbox:
    print("Getting API sandbox key...")
    api_key = open("API_sandbox.key", "r").read()
else:
    print("Getting API key...")
    api_key = open("API.key", "r").read()
    
if sandbox:
    url_tns_api="https://sandbox-tns.weizmann.ac.il/api"
else:
    url_tns_api="https://wis-tns.weizmann.ac.il/api"
    
if not sandbox:
    print("WARNING: submitting to real (not sandbox) TNS website")

Getting API key...


# Prepare TNS report

# CHECK AUTHOR LIST!!!
(key is used to query only at this step)

In [16]:
reporter = "G. Pignata, F.E. Bauer, F. Forster, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, F.E. Bauer, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"
report = {"at_report": {}}
counter = 0
for oid in candidates:
    #if oid not in test:#["ZTF19acgjpfd", "ZTF18acszatf", "ZTF18acrkmnm"]:
    #    continue
    #if oid == "ZTF18aamtgd": #ZTF19acymtgr": #ZTF19acnqqhl":#in ["ZTF19acmdwjy", "ZTF19acmdwjz"]:
    #    continue
    #if oid == "ZTF20aahofsn":
    #    continue
    print("\n", counter, oid)
    sn_report = alerce.do_TNS_report(api_key, oid, reporter, verbose=False, test=False)
    if sn_report:
        report["at_report"]["%i" % counter] = sn_report
        counter += 1



 0 ZTF20aakoeur
http://alerce.online/object/ZTF20aakoeur


Early SN candidate (r-rise > 0.11+-0.03 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20aakoeur 
Astronomical transient is known: [{'objname': '2020bkt', 'prefix': 'AT'}] 


 0 ZTF20aakodiq
http://alerce.online/object/ZTF20aakodiq


Early SN candidate (g-rise > 0.18+-0.06 mag/day, r-rise > 0.14+-0.06 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20aakodiq 

 1 ZTF20aaknzsj
http://alerce.online/object/ZTF20aaknzsj


Early SN candidate (g-rise > 0.68+-0.07 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20aaknzsj 

 2 ZTF20aakbtyz
http://alerce.online/object/ZTF20aakbtyz


Early SN candidate (g-rise > 0.18+-0.02 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20aakbtyz 
Astronomical transient is known: [{'objname': '2020bjg', 'prefix': 'AT'}] 


 2 ZTF20aakmbql
http://alerce.online/object/ZTF20aakmbql


Early SN candidate (r-rise > 0.14+-0.05 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20aakmbql 

 3 ZTF20aakocfl
http://alerce.online/object/ZTF20aakocfl


Early SN candidate (g-rise > 0.20+-0.10 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20aakocfl 

 4 ZTF20aakoenh
http://alerce.online/object/ZTF20aakoenh


Early SN candidate (g-rise > 0.23+-0.16 mag/day, r-rise > 0.40+-0.11 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20aakoenh 

 5 ZTF20aaknyfo
http://alerce.online/object/ZTF20aaknyfo


Early SN candidate (g-rise > 1.00 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20aaknyfo 


# Manual modifications

In [17]:
#report['at_report']['3']['remarks'] = 'Nova candidate in M31 (g-rise > 0.16+-0.01 mag/day, r-rise > 0.24+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxrihd '

# Display report

In [18]:
print(json.dumps(report, indent=4, sort_keys=True))

{
    "at_report": {
        "0": {
            "at_type": "1",
            "dec": {
                "error": "0.09",
                "units": "arcsec",
                "value": "24.0622016"
            },
            "discovery_data_source_id": "48",
            "discovery_datetime": "2020-01-27 11:56:06.003",
            "host_name": "SDSS J143816.31+240341.8",
            "host_redshift": "0.109",
            "internal_name": "ZTF20aajcpyy",
            "non_detection": {
                "comments": "Data provided by ZTF",
                "exptime": "30",
                "filter_value": "111",
                "flux_units": "1",
                "instrument_value": "196",
                "limiting_flux": "20.2996006011963",
                "obsdate": "2020-01-24 13:27:09.999",
                "observer": "Robot"
            },
            "photometry": {
                "photometry_group": {
                    "0": {
                        "comments": "Data provided by ZTF",
       

# Save report

In [19]:
json_report = '%s/%s.json' % (candidates_dir, refstring)
with open(json_report, 'w') as outfile:
    json.dump(report, outfile)

# Send to TNS

**WARNING: key is used to send candidates!**

In [20]:
json_report

'candidates/20200127.json'

In [21]:
r = alerce.send_json_report(api_key, url_tns_api, json_report)

# Show output

In [22]:
r.content

b'{"id_code":200,"id_message":"OK","data":{"report_id":32884,"recieved_data":{"at_report":{"0":{"ra":{"value":219.5676203,"error":0.09,"units":"arcsec"},"dec":{"value":24.0622016,"error":0.09,"units":"arcsec"},"reporting_group_id":74,"discovery_data_source_id":48,"reporter":"F. Forster, G. Pignata, F.E. Bauer, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, F.E. Bauer, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker","discovery_datetime":"2020-01-27 11:56:06.003","at_type":1,"internal_name":"ZTF20aajcpyy","remarks":"SN candidate classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http:\\/\\/alerce.online\\/object\\/ZTF20aajcpyy ","non_detection":{"obsdate":"2020-01-24 13:27:09.999","limiting_flux":20.299600601196,"flux_units":1,"filter_value":111,"instrument_value":196,"exptime":30,"observer":"Robot","c

# Check the following link and confirm that the latest SNe are from the reported day

https://wis-tns.weizmann.ac.il/search?&discovered_period_value=1&discovered_period_units=days&reporting_groupid[]=74&discovery_data_source_id=48&num_page=50&format=html&edit[type]=&edit[objname]=&edit[id]=&display[redshift]=1&display[hostname]=1&display[host_redshift]=1&display[source_group_name]=1&display[classifying_source_group_name]=1&display[discovering_instrument_name]=0&display[classifing_instrument_name]=0&display[programs_name]=0&display[internal_name]=1&display[isTNS_AT]=0&display[public]=1&display[end_pop_period]=0&display[spectra_count]=1&display[discoverymag]=1&display[discmagfilter]=1&display[discoverydate]=1&display[discoverer]=1&display[remarks]=0&display[sources]=0&display[bibcode]=0&display[ext_catalogs]=0&sort=desc&order=discoverydate